<a href="https://colab.research.google.com/github/VasantiP/fastai/blob/main/bunnies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gradio Bunnies

In [ ]:
!pip install -Uqq fastai


In [ ]:
from fastai.vision.all import *

In [ ]:
from duckduckgo_search import DDGS #DuckDuckGo
def search_images(keywords, max_images=200): 
    return L(DDGS().images(keywords=keywords, max_results=max_images)).itemgot('image')

In [ ]:
urls = search_images('bunny photos', max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
dest = 'bunny.jpg'
download_url(urls[0], dest, show_progress=False)

im = Image.open(dest)
im.to_thumb(256, 256)

In [ ]:
download_url(search_images('llama photos', max_images=1)[0], 'llama.jpg', show_progress=False)
Image.open('llama.jpg').to_thumb(256, 256)

In [ ]:
searches = 'bunny', 'llama'
path = Path('bunny_or_not')

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    time.sleep(5)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
# remove any photos that do not download correctly
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = ImageDataLoaders.from_path_func(path=path,
    fnames=get_image_files(path), valid_pct=0.2, seed=42,
    label_func=parent_label,
    item_tfms=[Resize(192, method='squish')])

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
is_bunny,_,probs = learn.predict(PILImage.create('bunny.jpg'))
print(f'This is a : {is_bunny}.')
print(f"Probability it's a bird: {probs[0]:.4f}")

In [ ]:
learn.export('model.pkl')